In [1]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
%%time
# optimize to pull one query by wire centers

# multigig enabled polygons:
# PHNXAZNE - polygon 1179499 - multigig enabled 9/30/22
# PHNXAZSO - polygons 1179547, 1179498, 1179121 - multigig enabled 9/30/22
# ARVDCOMA - polygon 1178832 
# MPLSMNBE - polygon 1178694
# PYLPWA01 - polygon 1178144
# GLDNCOMA - polygons 1179687, 1179674 - enabled 10/10/22

# each polygon will be associated with only one wire center, so only polygon id is needed in the filter

multigig_enabled_polygons_query = """

 WITH LQO AS (
            SELECT  DISTINCT LUID
                    ,WIRE_CENTER_ID
                    ,ADDRESS
                    ,ZIP_CODE
                    ,UNIT
                    ,UNI_POLYGONID        
            
            FROM    LQSMO.LU_QUAL_O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND (UNI_POLYGONID in ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1178694', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674' --GLDNCOMA
                                          )
                        OR WIRE_CENTER_ID = 'MPLSMNBE') -- per request, want all polygons in MPLSMNBE
        
        ),
        LQ AS (
        
        SELECT  Q.LUID
                ,Q.WIRE_CENTER_ID
                ,Q.ADDRESS
                ,Q.HNO
                ,Q.ST
                ,Q.UNIT
                ,Q.FLR
                ,Q.BLDG
                ,Q.COM
                ,Q.ZIP_CODE
                ,Q.TAR_CODE
                ,Q.BORN_ON_DATE
                ,Q.CAF_CENSUS_BLOCK
                ,Q.SERVING_TERMINAL
                ,Q.LU_XBOX
                ,Q.ASSOCIATED_TERMINAL_ID
                ,Q.REMOTE_TERMINAL_ID
                ,Q.REMOTE_TYPE
                ,Q.RES_BUS
                ,Q.LU_STAT

            FROM
                LU_QUAL Q
                JOIN LQO O
                    ON Q.WIRE_CENTER_ID = O.WIRE_CENTER_ID
        ),
        
        LC AS (
            
            SELECT * 
            FROM LU_CIRCUITS
        )
        
        SELECT DISTINCT
                O.LUID AS LU_QUAL_O_LUID
                ,O.UNI_POLYGONID
                ,A.*
                ,COALESCE(B.TELEPHONE_NUMBER,LCO.SUBSCRIBER) as TELEPHONE_OR_SUBSCRIBER
                ,B.LOOP_NAME
                ,B.CIRCUIT_IDENTIFIER
                ,B.SERVICE_CODE
                ,B.SERVICE_TYPE
                ,B.CS_USOC
                ,COALESCE(B.AL_USOC,LCO.HSI_SERVICE_PROFILE) as AL_USOC_OR_HSI_SERVICEPROFILE
                ,B.USER_ID
                ,B.TIME_STAMP
        FROM  LQO O
                JOIN LQ A
                    ON (TRIM(A.ADDRESS)||NVL(A.UNIT||A.FLR||A.BLDG,'.')=O.ADDRESS||NVL(REGEXP_REPLACE(O.UNIT, '^UNIT\s|^UNT\s|^APT\s|^FLR\s|BLDG\s', ''),'.'))
                    AND O.WIRE_CENTER_ID = A.WIRE_CENTER_ID
                LEFT JOIN LC B
                    ON  A.WIRE_CENTER_ID=B.WIRE_CENTER_ID
                    AND A.LUID=B.LUID
                LEFT JOIN LU_CIRCUITS_O LCO
                    ON O.LUID = LCO.LUID
                    AND O.WIRE_CENTER_ID = LCO.WIRE_CENTER_ID
                    AND LCO.SERVICE_FACS_STATUS = 'WKG'   --working customer base

"""

multigig_enabled_polygons_df = pd.read_sql(multigig_enabled_polygons_query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 250 ms
Wall time: 3min 14s


In [5]:
# check unique wire centers
multigig_enabled_polygons_df['WIRE_CENTER_ID'].value_counts()

MPLSMNBE    2536
PHNXAZSO     886
GLDNCOMA     421
PYLPWA01     342
PHNXAZNE     218
ARVDCOMA     207
Name: WIRE_CENTER_ID, dtype: int64

In [7]:
# separate data by wire center - manually add in wire centers

MPLSMNBE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'MPLSMNBE']
PHNXAZSO = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZSO']
PYLPWA01 = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PYLPWA01']
PHNXAZNE = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'PHNXAZNE']
ARVDCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'ARVDCOMA']
GLDNCOMA = multigig_enabled_polygons_df[multigig_enabled_polygons_df['WIRE_CENTER_ID'] == 'GLDNCOMA']

In [10]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate + "_LU_QUAL_slowsales.xlsx"

# add in new sheets manually if there are new wire centers turning up

with pd.ExcelWriter(excelfilename) as writer:  
    ARVDCOMA.to_excel(writer, sheet_name='ARVDCOMA', index = False)
    MPLSMNBE.to_excel(writer, sheet_name='MPLSMNBE_ALL_POLYGONS', index = False)
    PYLPWA01.to_excel(writer, sheet_name='PYLPWA01', index = False)
    PHNXAZSO.to_excel(writer, sheet_name='PHNXAZSO', index = False)
    PHNXAZNE.to_excel(writer, sheet_name='PHNXAZNE', index = False)
    GLDNCOMA.to_excel(writer, sheet_name='GLDNCOMA', index = False)